In [1]:
import pandas as pd
import wrds
import os
import sqlite3
from pathlib import Path

path_to_db = os.path.join(Path(os.getcwd()).parent) + "/data/db/option_prices.db"
db_uri = "sqlite:///" + path_to_db

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection(wrds_username=os.getenv("WRDS_USERNAME"), wrds_password=os.getenv("WRDS_PASSWORD"))

Loading library list...
Done


In [3]:
# Connection to the local SQLite database
conn_sqlite = sqlite3.connect(path_to_db)

## Get SP500 constituents from CRSP

In [4]:
### Step 2: Get SP500 Index Membership from CRSP
### I opt for the monthly frequency of the data, 
### but one can choose to work with crsp.dsp500list 
### if more precise date range is needed.

sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.dsp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/1925'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [5]:
### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
### - You don't need this step if only PERMNO is required
### - This step aims to add TICKER, SHRCD, EXCHCD and etc. 

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]

In [6]:
### Step 4: Add Compustat Identifiers
### - Link with Compustat's GVKEY and IID if need to work with 
###   fundamental data
### - Linkage is done through crsp.ccmxpf_linktable

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno,
                  linktype, linkprim, linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]

# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', 'linktype', \
                                  'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip',\
                     'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]

In [7]:
### Step 5: Remove unnecessary rows and columns
### - Remove entries with none cusip or ticker 

sp500ccm = sp500ccm[['permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd',
       'ticker', 'gvkey', 'iid', 'start', 'ending']]
sp500ccm = sp500ccm.drop_duplicates()
sp500ccm = sp500ccm.sort_values(by=['comnam'])

#sp500ccm['start'] = pd.to_datetime(sp500ccm['start'], format='%Y-%m-%d')
#sp500ccm['ending'] = pd.to_datetime(sp500ccm['ending'], format='%Y-%m-%d')

sp500ccm = sp500ccm[(~sp500ccm['ncusip'].isna()) & (~sp500ccm['ticker'].isna())]

# data_types = {'permno': db.types.Integer(), 'comnam':db.types.VARCHAR(),
#               'ncusip': db.types.VARCHAR(), 'shrcd': db.types.Integer(),
#               'exchcd': db.types.Integer(), 'hsiccd': db.types.Integer(),
#               'ticker': db.types.VARCHAR(), 'gvkey': db.types.VARCHAR(), 
#               'iid': db.types.VARCHAR(), 'start': db.DateTime(),
#               'ending': db.DateTime()}

In [8]:
### Step 6: Retrieve Compustat cusips
gvkeys = list(sp500ccm.gvkey.unique())
gvkeys_str = ', '.join([f"'{x}'" for x in gvkeys])
compustat=conn.raw_sql(f"""select * from comp.names where gvkey in ({gvkeys_str})""")
compustat = compustat[['gvkey', 'conm', 'tic', 'cusip', 'cik', 'sic', 'naics', 'gsubind','gind']]


sp500full = pd.merge(sp500ccm, compustat, how='left', on=['gvkey'])

In [9]:
sp500full['permno'] = sp500full['permno'].astype(int)
sp500full['shrcd'] = sp500full['shrcd'].astype(int)
sp500full['exchcd'] = sp500full['exchcd'].astype(int)
sp500full['hsiccd'] = sp500full['hsiccd'].astype(int)
sp500full['start'] = sp500full['start'].astype(str)
sp500full['ending'] = sp500full['ending'].astype(str)

In [10]:
sp500full

,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,conm,tic,cusip,cik,sic,naics,gsubind,gind
0,76129,3COM CORP,88553510,11,3,3674,COMS,010553,01,1996-01-02,2000-07-27,3COM CORP,COMS.,885535104,0000738076,3576,334119,45201020,452010
1,22592,3M CO,88579Y10,11,1,3841,MMM,007435,01,1957-03-01,2023-12-29,3M CO,MMM,88579Y101,0000066740,9997,999977,20105010,201050
2,10006,A C F INDUSTRIES INC,00080010,10,1,3743,ACF,001010,01,1957-03-01,1984-07-18,ACF INDUSTRIES INC,4165A,00099V004,0000910627,3743,336510,20304010,203040
3,50906,A D C TELECOMMUNICATIONS INC,00088630,11,3,3661,ADCT,001013,01,1999-08-02,2007-06-29,ADC TELECOMMUNICATIONS INC,ADCT.1,000886309,0000061478,3661,334210,45201020,452010
4,50906,A D C TELECOMMUNICATIONS INC,00088610,11,3,3661,ADCT,001013,01,1999-08-02,2007-06-29,ADC TELECOMMUNICATIONS INC,ADCT.1,000886309,0000061478,3661,334210,45201020,452010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,89070,ZIMMER HOLDINGS INC,98956P10,11,1,3842,ZMH,144559,01,2001-08-07,2023-12-29,ZIMMER BIOMET HOLDINGS INC,ZBH,98956P102,0001136869,3842,339113,35101010,351010
2554,84129,ZIONS BANCORPORATION,98970110,11,3,6021,ZION,011687,01,2001-06-25,2023-12-29,ZIONS BANCORPORATION NA,ZION,989701107,0000109380,6020,522110,40101015,401010
2555,84129,ZIONS BANCORPORATION N A,98970110,11,3,6021,ZION,011687,01,2001-06-25,2023-12-29,ZIONS BANCORPORATION NA,ZION,989701107,0000109380,6020,522110,40101015,401010
2556,13788,ZOETIS INC,98978V10,11,1,2834,ZTS,013721,01,2013-06-24,2023-12-29,ZOETIS INC,ZTS,98978V103,0001555280,2834,325412,35202010,352020


In [11]:
sp500full.to_sql(name='sp500_constituents', con=conn_sqlite, if_exists='replace', index=False)


2558

In [13]:
pd.read_sql("select * from sp500_constituents", conn_sqlite)

,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,conm,tic,cusip,cik,sic,naics,gsubind,gind
0,76129,3COM CORP,88553510,11,3,3674,COMS,010553,01,1996-01-02,2000-07-27,3COM CORP,COMS.,885535104,0000738076,3576,334119,45201020,452010
1,22592,3M CO,88579Y10,11,1,3841,MMM,007435,01,1957-03-01,2023-12-29,3M CO,MMM,88579Y101,0000066740,9997,999977,20105010,201050
2,10006,A C F INDUSTRIES INC,00080010,10,1,3743,ACF,001010,01,1957-03-01,1984-07-18,ACF INDUSTRIES INC,4165A,00099V004,0000910627,3743,336510,20304010,203040
3,50906,A D C TELECOMMUNICATIONS INC,00088630,11,3,3661,ADCT,001013,01,1999-08-02,2007-06-29,ADC TELECOMMUNICATIONS INC,ADCT.1,000886309,0000061478,3661,334210,45201020,452010
4,50906,A D C TELECOMMUNICATIONS INC,00088610,11,3,3661,ADCT,001013,01,1999-08-02,2007-06-29,ADC TELECOMMUNICATIONS INC,ADCT.1,000886309,0000061478,3661,334210,45201020,452010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,89070,ZIMMER HOLDINGS INC,98956P10,11,1,3842,ZMH,144559,01,2001-08-07,2023-12-29,ZIMMER BIOMET HOLDINGS INC,ZBH,98956P102,0001136869,3842,339113,35101010,351010
2554,84129,ZIONS BANCORPORATION,98970110,11,3,6021,ZION,011687,01,2001-06-25,2023-12-29,ZIONS BANCORPORATION NA,ZION,989701107,0000109380,6020,522110,40101015,401010
2555,84129,ZIONS BANCORPORATION N A,98970110,11,3,6021,ZION,011687,01,2001-06-25,2023-12-29,ZIONS BANCORPORATION NA,ZION,989701107,0000109380,6020,522110,40101015,401010
2556,13788,ZOETIS INC,98978V10,11,1,2834,ZTS,013721,01,2013-06-24,2023-12-29,ZOETIS INC,ZTS,98978V103,0001555280,2834,325412,35202010,352020


## Linking SP500 constituents from CRSP with OptionMetrics

The best practice when merging databases is to use primary identifiers. CRSP's is PERMNO and OptionMetrics security identifier is SECID. The historical mapping between SECID and CUSIP can be reconstructed based on the Security Name table (secnmd). The corresponding historical CUSIP in CRSP is NCUSIP and the history is located in the Stock Names table (stocknames).

The linking is available as `wrdsapps.opcrsphist` and we will use it here.

In [ ]:
link = db.raw_sql("""select * from wrdsapps.opcrsphist""")

scores | Description 
--- | ---
1 | 8-digit cusip historical matching
2 | historical ticker matching plus 6-digit cusips and similar company names (spedis < 30)
3 | historical ticker matching plus 6-digit cusips and company names are different
4 | historical ticker matching and similar company name (spedis < 30) but 6-digit cusips are different
5 | historical ticker matching but 6-digit cusips and company names are different
6 | no matching



In [ ]:
link.drop(columns=['score'], inplace=True)
link = link[~link['permno'].isna()]

In [ ]:
link

,secid,sdate,edate,permno
0,5001,1996-01-02,1996-03-13,10074
1,5002,1996-01-01,1996-02-22,10154
3,5004,1996-01-01,2000-01-27,80071
4,5005,1996-01-01,1997-08-12,85041
5,5006,1996-01-01,1996-08-28,10496
...,...,...,...,...
118272,218323,2023-03-16,2023-12-29,88885
118273,218324,2023-03-21,2023-12-29,23796
118274,218325,2023-03-17,2023-12-29,23814
118275,218326,2023-03-20,2023-12-29,23760


In [ ]:
# change types to int
link.loc[:, 'permno']  = link.loc[:,'permno'].astype(int)
link.loc[:, 'secid']   = link.loc[:,'secid'].astype(int)

In [ ]:
# write link to sqlite
link.to_sql('crsp_opm_link', conn_sqlite, if_exists='replace', index=False)

31477

In [14]:
conn_sqlite.close()
conn.close()